In [53]:
# import preprocessing libraries
import pandas as pd
import numpy as np
from tqdm import tqdm
import nltk
import json
import re
import csv
import ast
pd.set_option('display.max_colwidth', 300)

data = []

with open("booksummaries.txt", 'r', encoding='utf-8') as f:
    reader = csv.reader(f, dialect='excel-tab')
    for row in tqdm(reader):
        data.append(row)
book_index = []
book_id = []
book_author = []
book_name = []
summary = []
genre = []
genre_values = []
a = 1
for i in tqdm(data):
    book_index.append(a)
    a = a+1
    book_id.append(i[0])
    book_name.append(i[2])
    book_author.append(i[3])
    genre.append(i[5])
    summary.append(i[6])

df = pd.DataFrame({'Index': book_index, 'ID': book_id, 'BookTitle': book_name, 'Author': book_author,
                       'Genre': genre, 'Summary': summary})
df.head()

16559it [00:01, 13108.28it/s]
100%|██████████| 16559/16559 [00:00<00:00, 642659.34it/s]


,Index,ID,BookTitle,Author,Genre,Summary
0,1,620,Animal Farm,George Orwell,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"": ""Satire"", ""/m/0dwly"": ""Children's literature"", ""/m/014dfn"": ""Speculative fiction"", ""/m/02xlf"": ""Fiction""}","Old Major, the old boar on the Manor Farm, calls the animals on the farm for a meeting, where he compares the humans to parasites and teaches the animals a revolutionary song, 'Beasts of England'. When Major dies, two young pigs, Snowball and Napoleon, assume command and turn his dream into a p..."
1,2,843,A Clockwork Orange,Anthony Burgess,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""Novella"", ""/m/014dfn"": ""Speculative fiction"", ""/m/0c082"": ""Utopian and dystopian fiction"", ""/m/06nbt"": ""Satire"", ""/m/02xlf"": ""Fiction""}","Alex, a teenager living in near-future England, leads his gang on nightly orgies of opportunistic, random ""ultra-violence."" Alex's friends (""droogs"" in the novel's Anglo-Russian slang, Nadsat) are: Dim, a slow-witted bruiser who is the gang's muscle; Georgie, an ambitious second-in-command; and..."
2,3,986,The Plague,Albert Camus,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fiction"", ""/m/0pym5"": ""Absurdist fiction"", ""/m/05hgj"": ""Novel""}","The text of The Plague is divided into five parts. In the town of Oran, thousands of rats, initially unnoticed by the populace, begin to die in the streets. A hysteria develops soon afterward, causing the local newspapers to report the incident. Authorities responding to public pressure order t..."
3,4,1756,An Enquiry Concerning Human Understanding,David Hume,,"The argument of the Enquiry proceeds by a series of incremental steps, separated into chapters which logically succeed one another. After expounding his epistemology, Hume explains how to apply his principles to specific topics. In the first section of the Enquiry, Hume provides a rough introdu..."
4,5,2080,A Fire Upon the Deep,Vernor Vinge,"{""/m/03lrw"": ""Hard science fiction"", ""/m/06n90"": ""Science Fiction"", ""/m/014dfn"": ""Speculative fiction"", ""/m/01hmnh"": ""Fantasy"", ""/m/02xlf"": ""Fiction""}","The novel posits that space around the Milky Way is divided into concentric layers called Zones, each being constrained by different laws of physics and each allowing for different degrees of biological and technological advancement. The innermost, the ""Unthinking Depths"", surrounds the galacti..."


In [54]:
# df.to_csv('booksummaries.csv', index=False)
df.isna().sum()

df = df.drop(df[df['Genre'] == ''].index)
df = df.drop(df[df['Summary'] == ''].index)


genres_cleaned = []
for i in df['Genre']:
    genres_cleaned.append(list(json.loads(i).values()))
df['Genres'] = genres_cleaned


In [55]:
df.head()

,Index,ID,BookTitle,Author,Genre,Summary,Genres
0,1,620,Animal Farm,George Orwell,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"": ""Satire"", ""/m/0dwly"": ""Children's literature"", ""/m/014dfn"": ""Speculative fiction"", ""/m/02xlf"": ""Fiction""}","Old Major, the old boar on the Manor Farm, calls the animals on the farm for a meeting, where he compares the humans to parasites and teaches the animals a revolutionary song, 'Beasts of England'. When Major dies, two young pigs, Snowball and Napoleon, assume command and turn his dream into a p...","[Roman à clef, Satire, Children's literature, Speculative fiction, Fiction]"
1,2,843,A Clockwork Orange,Anthony Burgess,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""Novella"", ""/m/014dfn"": ""Speculative fiction"", ""/m/0c082"": ""Utopian and dystopian fiction"", ""/m/06nbt"": ""Satire"", ""/m/02xlf"": ""Fiction""}","Alex, a teenager living in near-future England, leads his gang on nightly orgies of opportunistic, random ""ultra-violence."" Alex's friends (""droogs"" in the novel's Anglo-Russian slang, Nadsat) are: Dim, a slow-witted bruiser who is the gang's muscle; Georgie, an ambitious second-in-command; and...","[Science Fiction, Novella, Speculative fiction, Utopian and dystopian fiction, Satire, Fiction]"
2,3,986,The Plague,Albert Camus,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fiction"", ""/m/0pym5"": ""Absurdist fiction"", ""/m/05hgj"": ""Novel""}","The text of The Plague is divided into five parts. In the town of Oran, thousands of rats, initially unnoticed by the populace, begin to die in the streets. A hysteria develops soon afterward, causing the local newspapers to report the incident. Authorities responding to public pressure order t...","[Existentialism, Fiction, Absurdist fiction, Novel]"
4,5,2080,A Fire Upon the Deep,Vernor Vinge,"{""/m/03lrw"": ""Hard science fiction"", ""/m/06n90"": ""Science Fiction"", ""/m/014dfn"": ""Speculative fiction"", ""/m/01hmnh"": ""Fantasy"", ""/m/02xlf"": ""Fiction""}","The novel posits that space around the Milky Way is divided into concentric layers called Zones, each being constrained by different laws of physics and each allowing for different degrees of biological and technological advancement. The innermost, the ""Unthinking Depths"", surrounds the galacti...","[Hard science fiction, Science Fiction, Speculative fiction, Fantasy, Fiction]"
5,6,2152,All Quiet on the Western Front,Erich Maria Remarque,"{""/m/098tmk"": ""War novel"", ""/m/016lj8"": ""Roman \u00e0 clef""}","The book tells the story of Paul Bäumer, a German soldier who—urged on by his school teacher—joins the German army shortly after the start of World War I. Bäumer arrives at the Western Front with his friends and schoolmates (Tjaden, Müller, Kropp and a number of other characters). There they me...","[War novel, Roman à clef]"


In [56]:
def clean_summary(text):
    text = re.sub("\'", "", text)
    text = re.sub("[^a-zA-Z]"," ",text)
    text = ' '.join(text.split())
    text = text.lower()
    return text

df['clean_summary'] = df['Summary'].apply(lambda x: clean_summary(x))
df.head()

,Index,ID,BookTitle,Author,Genre,Summary,Genres,clean_summary
0,1,620,Animal Farm,George Orwell,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"": ""Satire"", ""/m/0dwly"": ""Children's literature"", ""/m/014dfn"": ""Speculative fiction"", ""/m/02xlf"": ""Fiction""}","Old Major, the old boar on the Manor Farm, calls the animals on the farm for a meeting, where he compares the humans to parasites and teaches the animals a revolutionary song, 'Beasts of England'. When Major dies, two young pigs, Snowball and Napoleon, assume command and turn his dream into a p...","[Roman à clef, Satire, Children's literature, Speculative fiction, Fiction]",old major the old boar on the manor farm calls the animals on the farm for a meeting where he compares the humans to parasites and teaches the animals a revolutionary song beasts of england when major dies two young pigs snowball and napoleon assume command and turn his dream into a philosophy t...
1,2,843,A Clockwork Orange,Anthony Burgess,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""Novella"", ""/m/014dfn"": ""Speculative fiction"", ""/m/0c082"": ""Utopian and dystopian fiction"", ""/m/06nbt"": ""Satire"", ""/m/02xlf"": ""Fiction""}","Alex, a teenager living in near-future England, leads his gang on nightly orgies of opportunistic, random ""ultra-violence."" Alex's friends (""droogs"" in the novel's Anglo-Russian slang, Nadsat) are: Dim, a slow-witted bruiser who is the gang's muscle; Georgie, an ambitious second-in-command; and...","[Science Fiction, Novella, Speculative fiction, Utopian and dystopian fiction, Satire, Fiction]",alex a teenager living in near future england leads his gang on nightly orgies of opportunistic random ultra violence alexs friends droogs in the novels anglo russian slang nadsat are dim a slow witted bruiser who is the gangs muscle georgie an ambitious second in command and pete who mostly pla...
2,3,986,The Plague,Albert Camus,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fiction"", ""/m/0pym5"": ""Absurdist fiction"", ""/m/05hgj"": ""Novel""}","The text of The Plague is divided into five parts. In the town of Oran, thousands of rats, initially unnoticed by the populace, begin to die in the streets. A hysteria develops soon afterward, causing the local newspapers to report the incident. Authorities responding to public pressure order t...","[Existentialism, Fiction, Absurdist fiction, Novel]",the text of the plague is divided into five parts in the town of oran thousands of rats initially unnoticed by the populace begin to die in the streets a hysteria develops soon afterward causing the local newspapers to report the incident authorities responding to public pressure order the colle...
4,5,2080,A Fire Upon the Deep,Vernor Vinge,"{""/m/03lrw"": ""Hard science fiction"", ""/m/06n90"": ""Science Fiction"", ""/m/014dfn"": ""Speculative fiction"", ""/m/01hmnh"": ""Fantasy"", ""/m/02xlf"": ""Fiction""}","The novel posits that space around the Milky Way is divided into concentric layers called Zones, each being constrained by different laws of physics and each allowing for different degrees of biological and technological advancement. The innermost, the ""Unthinking Depths"", surrounds the galacti...","[Hard science fiction, Science Fiction, Speculative fiction, Fantasy, Fiction]",the novel posits that space around the milky way is divided into concentric layers called zones each being constrained by different laws of physics and each allowing for different degrees of biological and technological advancement the innermost the unthinking depths surrounds the galactic core ...
5,6,2152,All Quiet on the Western Front,Erich Maria Remarque,"{""/m/098tmk"": ""War novel"", ""/m/016lj8"": ""Roman \u00e0 clef""}","The book tells the story of Paul Bäumer, a German soldier who—urged on by his school teacher—joins the German army shortly after the start of World War I. Bäumer arrives at the Western Front with his friends and schoolmates (Tjaden, Müller, Kropp and 

In [57]:
# numbers of books in df
len(df)

# drop the id and genre column from df
df = df.drop(['ID', 'Genre'], axis=1)
df.head()


,Index,BookTitle,Author,Summary,Genres,clean_summary
0,1,Animal Farm,George Orwell,"Old Major, the old boar on the Manor Farm, calls the animals on the farm for a meeting, where he compares the humans to parasites and teaches the animals a revolutionary song, 'Beasts of England'. When Major dies, two young pigs, Snowball and Napoleon, assume command and turn his dream into a p...","[Roman à clef, Satire, Children's literature, Speculative fiction, Fiction]",old major the old boar on the manor farm calls the animals on the farm for a meeting where he compares the humans to parasites and teaches the animals a revolutionary song beasts of england when major dies two young pigs snowball and napoleon assume command and turn his dream into a philosophy t...
1,2,A Clockwork Orange,Anthony Burgess,"Alex, a teenager living in near-future England, leads his gang on nightly orgies of opportunistic, random ""ultra-violence."" Alex's friends (""droogs"" in the novel's Anglo-Russian slang, Nadsat) are: Dim, a slow-witted bruiser who is the gang's muscle; Georgie, an ambitious second-in-command; and...","[Science Fiction, Novella, Speculative fiction, Utopian and dystopian fiction, Satire, Fiction]",alex a teenager living in near future england leads his gang on nightly orgies of opportunistic random ultra violence alexs friends droogs in the novels anglo russian slang nadsat are dim a slow witted bruiser who is the gangs muscle georgie an ambitious second in command and pete who mostly pla...
2,3,The Plague,Albert Camus,"The text of The Plague is divided into five parts. In the town of Oran, thousands of rats, initially unnoticed by the populace, begin to die in the streets. A hysteria develops soon afterward, causing the local newspapers to report the incident. Authorities responding to public pressure order t...","[Existentialism, Fiction, Absurdist fiction, Novel]",the text of the plague is divided into five parts in the town of oran thousands of rats initially unnoticed by the populace begin to die in the streets a hysteria develops soon afterward causing the local newspapers to report the incident authorities responding to public pressure order the colle...
4,5,A Fire Upon the Deep,Vernor Vinge,"The novel posits that space around the Milky Way is divided into concentric layers called Zones, each being constrained by different laws of physics and each allowing for different degrees of biological and technological advancement. The innermost, the ""Unthinking Depths"", surrounds the galacti...","[Hard science fiction, Science Fiction, Speculative fiction, Fantasy, Fiction]",the novel posits that space around the milky way is divided into concentric layers called zones each being constrained by different laws of physics and each allowing for different degrees of biological and technological advancement the innermost the unthinking depths surrounds the galactic core ...
5,6,All Quiet on the Western Front,Erich Maria Remarque,"The book tells the story of Paul Bäumer, a German soldier who—urged on by his school teacher—joins the German army shortly after the start of World War I. Bäumer arrives at the Western Front with his friends and schoolmates (Tjaden, Müller, Kropp and a number of other characters). There they me...","[War novel, Roman à clef]",the book tells the story of paul b umer a german soldier who urged on by his school teacher joins the german army shortly after the start of world war i b umer arrives at the western front with his friends and schoolmates tjaden m ller kropp and a number of other characters there they meet stani...


In [58]:
print(len(df))

12841


In [59]:
# import pickle
import pickle 

with open('pickled_data/book_descriptions.pickle', 'rb') as f:
    book_descriptions = pickle.load(f)
    
print(len(book_descriptions))

5607


In [ ]:
# unpickle book_genres
import pickle

with open('pickled_data/book_genres.pickle', 'rb') as f:
    book_genres = pickle.load(f)


# convert each element in book_genres to a list of genres
book_genre_lists = []
for i in book_genres:
    # seperate by space
    i = i.split()
    book_genre_lists.append(i)
print((book_genre_lists))



In [ ]:
# unpickle book_titles

with open('pickled_data/book_titles.pickle', 'rb') as f:
    book_titles = pickle.load(f)
print(book_titles)

In [ ]:
# remove line space from book titles

book_titles_cleaned = []
for i in book_titles:
    # separate by space
    i = i.split()
    # remove first element if number
    if i[0].isdigit():
        i.pop(0)
    # reemove last element if line spac
    if i[-1] == '\n':
        i.pop(-1)
    
    #convert i back to string
    i = ' '.join(i)
    book_titles_cleaned.append(i)
print(book_titles_cleaned)


In [ ]:
import pickle 

with open ('pickled_data/book_authors.pickle', 'rb') as f:
    book_authors = pickle.load(f)

book_authors_cleaned = []
# remove the first token in each element in book_authors
for i in book_authors:
    # convert i to string
    i = str(i)
    i = i.split()
    # if the first token is a number, then remove it
    if i[0].isdigit():
        i.pop(0)
    book_authors_cleaned.append(i)

#convert each element in book_authors_cleaned to a string
book_authors_cleaned_string = []
for i in book_authors_cleaned:
    i = ' '.join(i)
    book_authors_cleaned_string.append(i)

book_authors_cleaned = book_authors_cleaned_string
print(book_authors_cleaned)

In [64]:
# append book_authors_cleaned, book_genres, book_titles, and book_descriptions to df
import pandas as pd
#create dataframe with book_authors_cleaned, book_genres, book_titles, and book_descriptions
df2 = pd.DataFrame({'BookTitle': book_titles_cleaned, 'Author': book_authors_cleaned, 'Genres': book_genre_lists, 'clean_summary': book_descriptions})
df2.head()

# add index column to df2 starting from 12841
df2['Index'] = range(12841, 12841+len(df2))
df2.head()

,BookTitle,Author,Genres,clean_summary,Index
0,The Fault in Our Stars,John Green,"[Young_Adult, Romance, Fiction, Contemporary, Realistic_Fiction]","Despite the tumor-shrinking medical miracle that has bought her a few years, Hazel has never been anything but terminal, her final chapter inscribed upon diagnosis. But when a gorgeous plot twist named Augustus Waters suddenly appears at Cancer Kid Support Group, Hazel's story is about to be com...",12841
1,"Divergent (Divergent, #1)",Veronica Roth,"[Young_Adult, Dystopia, Fiction, Fantasy, Science_Fiction]","In Beatrice Prior's dystopian Chicago world, society is divided into five factions, each dedicated to the cultivation of a particular virtue—Candor (the honest), Abnegation (the selfless), Dauntless (the brave), Amity (the peaceful), and Erudite (the intelligent). On an appointed day of every ye...",12842
2,"Mockingjay (The Hunger Games, #3)",Suzanne Collins,"[Young_Adult, Science_Fiction, Dystopia, Fiction, Fantasy]","My name is Katniss Everdeen.Why am I not dead?I should be dead.Katniss Everdeen, girl on fire, has survived, even though her home has been destroyed. Gale has escaped. Katniss's family is safe. Peeta has been captured by the Capitol. District 13 really does exist. There are rebels. There are new...",12843
3,Gone Girl,Gillian Flynn,"[Fiction, Mystery, Thriller, Mystery_Thriller, Crime]","Who are you?What have we done to each other?These are the questions Nick Dunne finds himself asking on the morning of his fifth wedding anniversary when his wife Amy suddenly disappears. The police suspect Nick. Amy's friends reveal that she was afraid of him, that she kept secrets from him. He ...",12844
4,The Martian,Andy Weir,"[Science_Fiction, Fiction, Audiobook, Adventure, Space]","Six days ago, astronaut Mark Watney became one of the first people to walk on Mars. Now, he’s sure he’ll be the first person to die there.After a dust storm nearly kills him and forces his crew to evacuate while thinking him dead, Mark finds himself stranded and completely alone with no way to e...",12845


In [65]:
# combine df and df2
combined_df = pd.concat([df, df2], ignore_index=True)

In [66]:
print(len(combined_df))

18448


In [67]:
combined_df.head()

,Index,BookTitle,Author,Summary,Genres,clean_summary
0,1,Animal Farm,George Orwell,"Old Major, the old boar on the Manor Farm, calls the animals on the farm for a meeting, where he compares the humans to parasites and teaches the animals a revolutionary song, 'Beasts of England'. When Major dies, two young pigs, Snowball and Napoleon, assume command and turn his dream into a p...","[Roman à clef, Satire, Children's literature, Speculative fiction, Fiction]",old major the old boar on the manor farm calls the animals on the farm for a meeting where he compares the humans to parasites and teaches the animals a revolutionary song beasts of england when major dies two young pigs snowball and napoleon assume command and turn his dream into a philosophy t...
1,2,A Clockwork Orange,Anthony Burgess,"Alex, a teenager living in near-future England, leads his gang on nightly orgies of opportunistic, random ""ultra-violence."" Alex's friends (""droogs"" in the novel's Anglo-Russian slang, Nadsat) are: Dim, a slow-witted bruiser who is the gang's muscle; Georgie, an ambitious second-in-command; and...","[Science Fiction, Novella, Speculative fiction, Utopian and dystopian fiction, Satire, Fiction]",alex a teenager living in near future england leads his gang on nightly orgies of opportunistic random ultra violence alexs friends droogs in the novels anglo russian slang nadsat are dim a slow witted bruiser who is the gangs muscle georgie an ambitious second in command and pete who mostly pla...
2,3,The Plague,Albert Camus,"The text of The Plague is divided into five parts. In the town of Oran, thousands of rats, initially unnoticed by the populace, begin to die in the streets. A hysteria develops soon afterward, causing the local newspapers to report the incident. Authorities responding to public pressure order t...","[Existentialism, Fiction, Absurdist fiction, Novel]",the text of the plague is divided into five parts in the town of oran thousands of rats initially unnoticed by the populace begin to die in the streets a hysteria develops soon afterward causing the local newspapers to report the incident authorities responding to public pressure order the colle...
3,5,A Fire Upon the Deep,Vernor Vinge,"The novel posits that space around the Milky Way is divided into concentric layers called Zones, each being constrained by different laws of physics and each allowing for different degrees of biological and technological advancement. The innermost, the ""Unthinking Depths"", surrounds the galacti...","[Hard science fiction, Science Fiction, Speculative fiction, Fantasy, Fiction]",the novel posits that space around the milky way is divided into concentric layers called zones each being constrained by different laws of physics and each allowing for different degrees of biological and technological advancement the innermost the unthinking depths surrounds the galactic core ...
4,6,All Quiet on the Western Front,Erich Maria Remarque,"The book tells the story of Paul Bäumer, a German soldier who—urged on by his school teacher—joins the German army shortly after the start of World War I. Bäumer arrives at the Western Front with his friends and schoolmates (Tjaden, Müller, Kropp and a number of other characters). There they me...","[War novel, Roman à clef]",the book tells the story of paul b umer a german soldier who urged on by his school teacher joins the german army shortly after the start of world war i b umer arrives at the western front with his friends and schoolmates tjaden m ller kropp and a number of other characters there they meet stani...


In [68]:
# pickle combined_df
import pickle

with open('pickled_data/combined_df.pickle', 'wb') as f:
    pickle.dump(combined_df, f)

In [81]:
genre_to_text = []

for i in combined_df['Genres']:
    genre_text = "".join(g.strip().replace(" ", "_")+ " " for g in i)
    #lowercase
    # genre_text = genre_text.lower()
    genre_to_text.append(genre_text)

    

In [84]:
genre_to_text

#Count Vectorizer to convert genres to vector

from sklearn.feature_extraction.text import CountVectorizer
genre_vectorizer = CountVectorizer()

genre_vectors = genre_vectorizer.fit_transform(genre_to_text)
genre_names = genre_vectorizer.get_feature_names()
genre_vectors = genre_vectors.todense().tolist()



c:\Python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [85]:
genre_df = pd.DataFrame(genre_vectors, columns=genre_names)
genre_df


,11th_century,17th_century,19th_century,20th_century,21st_century,40k,_school_stories,absurdist_fiction,abuse,academia,...,x_men,yeti,young_adult,young_adult_contemporary,young_adult_fantasy,young_adult_literature,youth,zombie,zombies,zombies_in_popular_culture
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18443,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18444,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18445,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18446,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
